<a href="https://colab.research.google.com/github/user074/literatureReviewer/blob/main/literatureReviewer.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This requires a GPU to run since Im using nougat-ocr, which is really good one for academic papers!

In [1]:
!pip install openai tiktoken nougat-ocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import openai
import tiktoken

In [3]:
openai.api_key = ('PUT_YOUR_API_KEY_HERE')

I used CLIP to test. It is a really long paper...
[CLIP Paper](https://arxiv.org/abs/2103.00020)

In [4]:
from google.colab import files
uploaded = files.upload()

Saving 2103.00020.pdf to 2103.00020.pdf


# nougat-ocr doing it's magic

In [43]:
import os
from pathlib import Path

uploadfilename = next(iter(uploaded))
uploadfilePath = os.path.join("/content/", uploadfilename)

In [45]:
!nougat $uploadfilePath -o /content

2023-09-22 04:36:02.796777: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
  0% 0/12 [00:00<?, ?it/s]WARNING:root:Found repetitions in sample 3
INFO:root:Processing file /content/2103.00020.pdf with 48 pages
 75% 9/12 [03:11<01:15, 25.07s/it]WARNING:root:Found repetitions in sample 3
 92% 11/12 [03:58<00:24, 24.50s/it]WARNING:root:Found repetitions in sample 1
100% 12/12 [04:24<00:00, 22.05s/it]


In [16]:
filename = uploadfilename.split('.pdf')[0]+'.mmd'

# Support functions

In [10]:

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
  """Returns the number of tokens used by a list of messages."""
  try:
      encoding = tiktoken.encoding_for_model(model)
  except KeyError:
      encoding = tiktoken.get_encoding("cl100k_base")
  if model == "gpt-3.5-turbo-0613":  # note: future models may deviate from this
      num_tokens = 0
      for message in messages:
          num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
          for key, value in message.items():
              num_tokens += len(encoding.encode(value))
              if key == "name":  # if there's a name, the role is omitted
                  num_tokens += -1  # role is always required and always 1 token
      num_tokens += 2  # every reply is primed with <im_start>assistant
      return num_tokens
  else:
      raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
  See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")

In [26]:
#read text from file and convert to list of messages for openai api

def read_text_from_file(filename):
    with open(filename, 'r') as file:
        text = file.read()
    return text

def read_without_references(filename):
    with open(filename, 'r') as file:
        content = file.readlines()

    # Define a variable to store the final content without references
    final_content = []

    # Flag to indicate if the "References" section has started
    references_started = False

    for line in content:
        if "References" in line:
            references_started = True
        if not references_started:
            final_content.append(line)

    return ''.join(final_content)

def convert_text_to_messages(text, user_instructions=None):
    messages = [
    {"role": "system", "content": "You are an expert academic journal reviewer in CVPR, IEEE, and ACM.\
        You are reviewing a paper.\
        You need to be constructive to the authors. It is necessary to be critical, but avoid offending the authors.\
        Instead, suggest how they could make the paper better.\
        "},
    # {"role": "user", "content": "Help me to polish following text."},
    ]
    if user_instructions:
        messages.extend([{"role": "user", "content": user_instructions} ])

    messages.extend([{"role": "user", "content": text} ])
    return messages


def convert_messages_to_text(messages):
    text = ''
    for message in messages:
        if message['role'] == 'user':
            text += 'Human: ' + message['text'] + '\n'
        elif message['role'] == 'assistant':
            text += 'AI: ' + message['text'] + '\n'
    return text

def write_text_to_file(text, filename):
    with open(filename, 'w') as file:
        file.write(text)

def write_messages_to_file(messages, filename):
    write_text_to_file(convert_messages_to_text(messages), filename)

#count tokens in text
def num_tokens_from_text(text, model = "gpt-3.5-turbo-0613"):
    messages = convert_text_to_messages(text)
    return num_tokens_from_messages(messages, model)

#read the text file, then return a list of texts whereas each contains less than given number of tokens
def split_text_into_list(text, max_tokens=4096):
    messages = []
    message = ''
    for line in text.splitlines():
        message_token = num_tokens_from_messages(convert_text_to_messages(message))
        line_token = num_tokens_from_messages(convert_text_to_messages(line))
        if message_token + line_token < max_tokens/2:
            message += line + '\n'
        else:
            messages.append(message)
            message = line + '\n'
    messages.append(message)
    return messages

#call openai api to polish the text
def instruct_text(text,user_instructions, model = "gpt-3.5-turbo"):
    messages = convert_text_to_messages(text, user_instructions=user_instructions)
    response = openai.ChatCompletion.create(model=model, messages=messages)
    return response

#Polish an entire txt file then write the results into a new file
def summarize_text_file(filename, model = "gpt-3.5-turbo"):
    text = read_without_references(filename)
    paragraphs = split_text_into_list(text, max_tokens=4096)
    polished_text = ''
    user_instructions = "The paper is too long. Now you are summarizing it part to part. Provide a summary of given section. Make it shorter but keep all the technical details."
    for paragraph in paragraphs:
        result = instruct_text(paragraph, user_instructions, model)
        polished_text += result['choices'][0]["message"]["content"]
    return polished_text

def write_text_to_file(text, filename):
    with open(filename, 'w') as file:
        file.write(text)

def summarize_text_file_to_file(filename, outputFileName, model = "gpt-3.5-turbo"):
    text = summarize_text_file(filename, model)
    write_text_to_file(text, outputFileName)

## Cost Estimation

Estimate your cost before doing anything for free. This is a rough estimate. No API key required

In [18]:
def cost_estimation(filename):
    text = read_without_references(filename)
    messages = convert_text_to_messages(text)
    model = "gpt-3.5-turbo-0613"
    paragraphs = split_text_into_list(text, max_tokens=4096)
    estimated_cost = num_tokens_from_messages(messages, model) * (0.0015 + 0.002) / 1000
    print(f"{num_tokens_from_messages(messages, model)} prompt tokens counted.")
    print(f"Number of paragraphs to process: {len(paragraphs)}")
    print(f"Estimated cost using gpt-3.5-turbo: ${estimated_cost}")

In [19]:
cost_estimation(filename)

24957 prompt tokens counted.
Number of paragraphs to process: 14
Estimated cost using gpt-3.5-turbo: $0.08734950000000001


# Run the Summarizer

In [24]:
summarize_text_file_to_file(filename, 'condensed_paper.txt')

In [25]:
#check the total token count from the summarized version
cost_estimation('condensed_paper.txt')

5400 prompt tokens counted.
Number of paragraphs to process: 3
Estimated cost using gpt-3.5-turbo: $0.018900000000000004


Longer than 4096 tokens so I will do a double condensation

In [27]:
summarize_text_file_to_file('condensed_paper.txt', 'condensed_condensed_paper.txt')

In [29]:
cost_estimation('condensed_condensed_paper.txt')

1966 prompt tokens counted.
Number of paragraphs to process: 1
Estimated cost using gpt-3.5-turbo: $0.006881


In [30]:
#Polish an entire txt file then write the results into a new file
def instruct_text_file(filename, user_instructions, model = "gpt-3.5-turbo"):
    text = read_text_from_file(filename)
    paragraphs = split_text_into_list(text, max_tokens=4096)
    polished_text = ''
    for paragraph in paragraphs:
        result = instruct_text(paragraph, user_instructions, model)
        polished_text += result['choices'][0]["message"]["content"]
    return polished_text

Start summarize with some instructions

In [31]:
#Summary
summary_instructions = "Now you are reviewing the summarized version of the paper.\
Give a concise summary of the paper for the literature review. \
Answer following questions in the summary:\
What problem is addressed in the paper?\
Is it a new problem? If so, why does it matter? If not, why does it still matter?\
What is the key to the solution? What is the main contribution?\
Do the experiments sufficiently support the claims?"

summary = instruct_text_file('condensed_condensed_paper.txt', summary_instructions)

#strengths
strengths_instructions = "Now you are reviewing the summarized version of the paper.\
Give a concise statement of the strengths of the paper for the literature review. \
What about the paper provides value - - interesting ideas that are experimentally validated, an insightful organization of related work, new tools,\
impressive results. Most importantly, what can someone interested in the topic learn from the paper. What are the key contributions and why do they matter?\
Use bullet points to write the strengths statement."

strengths = instruct_text_file('condensed_condensed_paper.txt', strengths_instructions)

#weaknesses
weaknesses_instructions = "Now you are reviewing the summarized version of the paper.\
Give a concise statement of the weaknesses of the paper for the literature review. \
What detracts from the contributions -- Does the paper lack controlled experiments to validate the contributions?\
Are there misleading claims or technical errors? Is it possible to understand (and ideally reproduce) the method and experimental \
setups by reading the paper?What are the key contributions and why do they matter? What aspects of the paper most need improvement?\
Use bullet points to write the weaknesses statement."

weaknesses = instruct_text_file('condensed_condensed_paper.txt', weaknesses_instructions)

final_summary = '**Summary**'+'\n'+summary + '\n\n' + '**Strengths**'+'\n'+strengths + '\n\n' + '**Weaknesses**'+'\n'+weaknesses

with open('literature_review.txt', 'w') as file:
    file.write(final_summary)



# Results

In [32]:
print(final_summary)

**Summary**
In this section, the authors evaluate the zero-shot transfer capabilities of CLIP on various datasets. They compare its performance to a baseline logistic regression model trained on ResNet-50 features. The results show that CLIP performs well on general object classification tasks but struggles on specialized tasks. The authors suggest using CLIP as a prior for few-shot learning to improve its performance. They analyze the estimated data efficiency of zero-shot transfer and find that it varies widely across datasets. The experiments provide support for the claims made by the authors.

In the evaluation of CLIP on the FairFace dataset, the authors compare the performance of zero-shot CLIP and logistic regression CLIP to other models. LR CLIP achieves higher accuracy on the FairFace dataset. The authors also test the model's performance across intersectional race and gender categories and find that it achieves high accuracy on gender classification for all race categories. H